In [1]:
import argparse
import time
import csv
import pickle
import operator
import datetime
import os

In [17]:
dataset = 'test_yelp_srgnn.csv'
with open(dataset, "r") as f:
    reader = csv.DictReader(f, delimiter='\t')
    print(reader)
    sess_clicks = {}
    sess_date = {}
    ctr = 0
    curid = -1
    curdate = None
    for data in reader:
        sessid = data['session_id']
        # 新的session
        if curdate and not curid == sessid:
            date = time.mktime(time.strptime(curdate[:19], '%Y-%m-%d %H:%M:%S'))
            sess_date[curid] = date # 记录最后一个时间
        curid = sessid # 新session
        item = data['item_id'], int(data['timeframe'])# item,time
        curdate = ''
        curdate = data['eventdate'] #取时间
        if sessid in sess_clicks:
            sess_clicks[sessid] += [item]
        else:
            sess_clicks[sessid] = [item] # 变成item列表
        ctr += 1
    date = time.mktime(time.strptime(curdate[:19], '%Y-%m-%d %H:%M:%S'))
    for i in list(sess_clicks):
        sorted_clicks = sorted(sess_clicks[i], key=operator.itemgetter(1))
        sess_clicks[i] = [c[0] for c in sorted_clicks] # 用时间排序
    sess_date[curid] = date # 最后一个
print("-- Reading data @ %ss" % datetime.datetime.now())


-- Reading data @ 2022-03-31 17:05:38.025606s


In [13]:
dates = list(sess_date.items())
tra_sess = sorted(dates, key=operator.itemgetter(1)) 

In [14]:
print(len(tra_sess))    # 186670    # 7966257
print(tra_sess)

24306
[('31603', 1514707696.0), ('9551', 1514708282.0), ('15563', 1514709573.0), ('3311', 1514710767.0), ('17826', 1514712979.0), ('34247', 1514713259.0), ('28264', 1514717495.0), ('26307', 1514718267.0), ('8854', 1514719241.0), ('4679', 1514720574.0), ('11246', 1514722162.0), ('28541', 1514757897.0), ('15072', 1514762525.0), ('3244', 1514764718.0), ('19474', 1514772679.0), ('29782', 1514776027.0), ('7558', 1514777952.0), ('4692', 1514783575.0), ('1458', 1514787774.0), ('25565', 1514796505.0), ('3266', 1514802894.0), ('26671', 1514803954.0), ('20061', 1514806449.0), ('14803', 1514807843.0), ('16486', 1514808442.0), ('9518', 1514814510.0), ('8344', 1514824856.0), ('31410', 1514856573.0), ('18147', 1514863059.0), ('34071', 1514863170.0), ('6956', 1514865944.0), ('25091', 1514867280.0), ('26309', 1514868602.0), ('19557', 1514874002.0), ('17986', 1514886753.0), ('30592', 1514887178.0), ('3577', 1514895475.0), ('8598', 1514896932.0), ('3546', 1514899816.0), ('1417', 1514905476.0), ('22468',

In [15]:
item_dict = {}
def obtian_tra():
    train_ids = []
    train_seqs = []
    train_dates = []
    item_ctr = 1
    for s, date in tra_sess:
        seq = sess_clicks[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
        if len(outseq) < 5:  # Doesn't occur
            continue
        train_ids += [s]
        train_dates += [date]
        train_seqs += [outseq]
    print(item_ctr)     # 43098, 37484
    return train_ids, train_dates, train_seqs

In [16]:
tra_ids, tra_dates, tra_seqs = obtian_tra()

44643


In [23]:
def process_seqs(iseqs, idates):
    out_seqs = []
    out_dates = []
    labs = []
    ids = []
    flag=0
    for id, seq, date in zip(range(len(iseqs)), iseqs, idates):
        
        endd=min(len(seq)-4,2)
        beginn=max(0,len(seq)-endd-99)
        for i in range(1, endd):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[beginn:-i]]
            out_dates += [date]
            ids += [id]
            print(len(out_seqs[-1]))
    return out_seqs, out_dates, labs, ids

In [28]:
tr_seqs, tr_dates, tr_labs, tr_ids = process_seqs(tra_seqs, tra_dates)

13
11
10
20
13
8
5
11
15
16
10
14
13
11
9
12
36
15
27
34
15
28
22
6
11
13
33
9
10
11
33
11
11
8
18
8
14
15
24
19
9
12
6
9
12
11
11
9
11
6
10
7
10
21
16
13
21
21
42
11
9
10
9
7
44
10
22
16
5
24
30
21
11
5
10
19
15
9
21
8
57
8
21
26
14
17
6
8
24
7
19
15
11
19
19
28
10
8
41
7
11
11
6
14
5
10
18
17
63
8
9
12
35
9
7
10
13
18
10
11
20
18
16
6
24
10
24
11
12
14
27
22
12
5
7
7
20
9
11
10
9
11
8
10
24
6
33
9
51
54
21
10
24
33
9
9
16
7
10
7
21
16
11
9
11
24
8
11
7
9
29
19
7
8
11
12
10
21
13
7
14
13
12
10
27
9
14
8
8
11
39
91
15
10
8
23
10
9
11
9
16
30
11
8
11
29
13
8
12
10
14
11
12
10
30
36
31
29
14
7
26
12
8
12
6
9
14
9
21
8
14
16
6
14
43
23
15
13
8
9
7
29
18
11
9
9
9
9
11
19
9
7
29
10
11
10
26
51
17
15
31
59
33
21
19
10
9
8
8
8
14
14
6
26
10
5
13
13
69
9
7
7
9
12
10
33
11
18
100
21
15
9
9
28
17
13
13
10
15
9
8
8
11
16
33
11
28
11
9
14
8
100
7
15
9
8
38
19
8
8
7
16
8
12
6
8
9
9
16
15
12
11
11
8
11
10
19
8
9
5
9
9
21
13
6
14
12
15
14
9
10
11
27
12
7
9
15
7
14
12
14
27
15
36
19
15
18
9
10
10
5
6


12
7
5
35
8
8
10
6
68
10
9
10
11
7
18
10
7
11
6
7
29
9
8
16
7
9
8
12
5
10
13
6
6
25
9
10
11
20
13
8
24
19
10
8
11
33
26
9
13
9
13
17
12
8
35
11
8
11
15
7
8
7
21
5
20
11
6
15
10
19
23
14
9
11
17
7
10
9
10
7
50
47
15
5
10
18
12
8
19
53
9
11
9
10
8
34
32
13
8
14
11
14
10
9
15
11
11
12
12
9
12
9
8
9
11
9
10
11
29
7
9
18
8
9
5
31
9
13
9
15
14
17
15
41
11
8
13
9
14
18
9
6
15
13
44
9
10
28
7
20
5
11
33
7
6
15
14
12
7
40
48
8
15
18
14
9
11
8
66
20
9
6
19
10
8
12
10
7
8
89
14
14
8
13
8
7
12
8
12
8
20
6
9
12
9
10
41
10
20
11
9
10
25
20
9
39
7
12
9
8
15
9
12
11
10
10
9
7
20
35
7
8
12
10
32
7
13
9
9
6
12
20
24
10
11
8
21
14
60
7
7
16
6
100
10
18
9
11
9
5
6
24
11
15
16
13
6
88
10
8
12
15
10
11
13
6
9
15
7
6
15
5
9
9
50
11
19
14
12
100
28
9
5
25
10
8
8
14
10
8
11
6
9
11
10
6
9
20
10
11
16
11
9
15
8
7
11
10
8
15
5
11
9
8
14
11
8
10
8
9
9
9
10
10
9
9
5
20
8
10
9
6
8
10
21
11
8
12
14
8
8
10
12
6
17
8
52
7
12
15
19
6
16
13
100
19
56
36
8
27
12
16
9
9
11
15
28
10
11
14
9
17
12
9
15
10
7
8
15
22
8
6
61
8


8
10
9
37
35
11
11
10
34
22
9
15
10
8
26
15
10
53
8
10
47
9
31
15
10
15
14
24
12
13
6
11
11
12
8
9
27
10
14
10
10
39
30
7
10
18
14
20
6
16
12
11
10
9
8
9
14
18
13
11
16
12
15
9
20
21
15
13
23
21
16
25
9
60
19
9
14
7
33
9
9
9
15
9
8
9
9
13
22
10
9
11
7
7
35
26
9
6
21
15
25
17
10
7
8
5
9
8
9
5
13
16
10
7
15
13
16
16
10
21
7
10
10
16
17
10
9
8
11
11
12
7
9
23
6
63
64
10
10
11
13
12
15
72
7
18
8
16
17
16
20
10
11
16
69
30
18
12
14
31
20
5
8
15
9
10
7
15
7
20
9
42
9
8
31
19
8
8
7
26
10
8
12
10
12
18
8
40
12
15
16
20
100
48
16
9
24
78
10
10
8
33
10
14
17
32
26
11
6
12
14
12
22
11
7
41
22
8
11
17
14
63
34
15
18
41
19
9
55
23
11
11
9
11
11
31
34
18
9
20
11
8
10
22
9
46
10
25
25
10
19
18
33
25
10
10
14
16
9
30
14
10
14
8
15
9
7
15
40
97
33
7
13
42
19
13
11
28
9
8
16
8
11
12
20
7
11
19
55
21
19
6
25
6
14
15
8
10
25
13
22
11
8
12
5
7
21
9
12
19
8
15
50
15
28
9
11
12
9
26
13
10
19
21
51
78
16
12
85
11
80
11
10
19
7
15
10
31
24
6
17
57
7
10
10
9
9
21
8
9
17
13
21
8
13
9
12
36
12
20
6
10
5
7
24
5
14

34
30
31
8
13
14
15
61
17
11
23
44
100
23
21
8
28
10
11
9
9
12
9
6
36
12
10
24
8
7
11
12
39
9
11
10
13
21
23
10
14
12
13
24
12
31
27
27
18
16
60
11
5
7
56
16
6
10
8
12
16
52
9
11
6
31
5
5
8
8
5
14
9
10
8
7
15
10
14
8
14
25
11
7
40
21
7
9
19
31
8
10
10
12
12
12
23
14
21
29
14
12
7
9
8
14
19
16
8
23
18
12
15
9
9
22
43
8
8
17
8
7
8
8
11
8
9
23
54
29
8
12
15
11
6
9
8
21
11
9
35
11
11
9
9
14
7
17
18
62
6
19
9
16
42
33
39
16
7
70
12
5
24
10
8
9
15
7
8
7
33
16
7
20
7
9
6
57
8
8
93
16
68
37
20
27
9
6
13
23
6
14
18
22
12
9
45
45
13
39
7
25
10
9
10
11
25
10
15
22
8
11
9
14
9
9
9
10
33
11
21
42
7
8
9
10
10
10
11
11
16
51
8
27
11
8
8
12
30
18
17
23
9
22
15
17
15
100
18
9
27
16
7
11
9
35
11
7
24
11
8
7
10
9
9
7
8
18
40
10
37
9
10
8
52
10
9
6
24
10
100
8
42
44
31
30
8
11
33
9
22
9
7
10
57
39
11
23
11
11
10
9
12
10
25
15
8
9
9
11
5
15
51
10
18
9
12
8
30
11
17
8
11
9
15
10
10
11
48
8
16
8
28
24
12
9
8
32
8
9
57
10
9
10
18
8
16
12
10
89
18
34
8
20
11
7
9
9
9
12
20
22
10
7
61
59
7
9
12
9
48
13
6
17
10
9

35
11
14
8
15
7
11
6
19
37
17
10
42
18
16
7
14
10
32
26
10
57
14
43
13
36
58
10
47
11
7
100
9
10
10
9
71
51
16
10
6
14
10
6
9
25
14
5
40
30
38
6
7
7
92
20
6
51
9
6
42
11
7
9
12
9
34
9
16
12
10
10
14
7
9
15
9
15
74
9
9
28
10
13
54
27
8
8
27
36
17
24
7
10
9
9
8
8
29
23
8
43
50
8
80
8
17
40
14
33
41
8
7
27
9
8
24
8
25
15
7
7
10
14
8
10
46
8
31
29
8
14
23
11
10
11
16
23
49
8
59
37
54
8
40
13
15
45
8
43
7
11
11
23
22
9
20
21
8
15
19
22
69
7
25
56
16
14
19
21
6
8
11
26
9
7
9
10
14
21
6
11
10
10
22
11
8
9
22
39
17
7
25
10
11
23
45
8
7
11
47
5
8
9
6
22
9
40
12
8
13
16
21
13
9
14
11
10
9
29
9
39
49
19
6
10
13
46
6
31
11
22
16
7
6
11
66
9
7
8
22
9
18
12
39
11
24
13
7
11
8
8
15
26
8
30
9
8
19
9
8
9
18
13
71
10
23
8
52
22
12
5
9
19
11
37
30
16
11
43
12
10
13
8
26
31
13
22
8
21
10
27
13
14
23
22
9
9
14
8
12
17
6
40
59
37
11
22
9
13
11
17
10
11
20
7
31
37
9
7
9
28
10
9
10
14
12
8
10
21
8
7
12
29
9
19
13
7
38
14
9
22
16
9
10
11
9
9
8
6
37
8
11
31
54
5
10
10
12
9
22
9
9
31
5
10
10
6
7
31
9
8
12
5
8
9


16
21
9
9
8
8
6
12
7
9
8
21
8
12
21
9
8
7
20
18
9
13
9
36
16
9
11
7
19
19
12
15
9
12
13
11
16
10
13
9
11
20
15
20
15
36
12
34
7
9
23
8
30
12
10
23
7
9
12
18
16
11
14
32
33
9
16
7
34
13
14
25
20
26
22
9
10
78
17
21
14
12
8
18
37
8
28
9
22
10
75
11
49
11
7
22
16
11
10
13
24
14
100
8
9
11
5
14
9
10
8
15
31
73
8
7
21
18
8
81
53
33
8
28
15
14
28
11
12
18
10
18
21
14
15
14
15
13
11
8
25
7
29
27
9
21
9
8
72
31
19
13
8
23
18
12
10
11
15
23
16
6
14
8
15
18
74
10
48
17
16
20
11
12
12
9
19
13
9
100
10
11
45
13
40
24
12
16
11
11
10
13
9
18
10
8
10
39
10
6
20
8
19
33
14
23
13
10
27
9
11
10
27
52
11
7
22
16
20
21
39
8
8
30
8
9
25
14
18
40
17
37
22
21
10
10
11
47
19
23
9
14
12
13
9
18
28
10
11
19
14
11
13
12
34
57
13
93
18
7
14
22
6
14
7
99
22
5
10
13
11
78
100
11
10
14
14
13
35
14
11
8
25
10
8
17
12
8
6
98
14
22
9
11
11
33
22
76
11
8
13
45
12
11
12
19
15
8
13
8
12
9
10
8
10
14
29
22
27
9
27
15
28
9
13
7
32
14
13
78
11
9
8
14
10
9
61
10
29
7
5
100
28
22
40
11
20
11
8
50
9
9
13
8
8
12
9
10
47
5
34
15


In [29]:
len(tr_seqs)

24017

In [26]:
tra = (tr_seqs, tr_labs)
if not os.path.exists('sample_yelp'):
    os.makedirs('sample_yelp')
pickle.dump(tra, open('sample_yelp/test.txt', 'wb'))
pickle.dump(tra_seqs, open('sample_yelp/all_test_seq.txt', 'wb'))
print('Done.')

Done.


In [11]:
tra

([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
  [1, 2, 3, 4, 5, 6, 7, 8, 9],
  [1, 2, 3, 4, 5, 6, 7, 8],
  [1, 2, 3, 4, 5, 6, 7],
  [1, 2, 3, 4, 5, 6],
  [1, 2, 3, 4, 5],
  [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28],
  [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27],
  [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26],
  [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25],
  [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
  [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
  [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22],
  [12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
  [12, 13, 14, 15, 16, 17, 18, 19, 20],
  [12, 13, 14, 15, 16, 17, 18, 19],
  [12, 13, 14, 15, 16, 17, 18],
  [12, 13, 14, 15, 16, 17],
  [12, 13, 14, 15, 16],
  [29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47],
  [29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46],
  [29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 4